In [1]:
import numpy as np
import pandas as pd
import os
from rdkit import Chem
from rdkit.Chem import Draw
import shutil
import tqdm
import pybel
import tarfile

In [2]:
path = "/aloy/home/acomajuncosa/MurD/GitHub/osa-murd/data/generated/zairachem"
smiles = []

for file in sorted(os.listdir(path)):
    df = pd.read_csv(os.path.join(path, file), header=None, names=['smiles'])['smiles'].tolist()
    smiles.extend(df)
    
print(len(smiles))
print(len(set(smiles)))

smiles = np.array_split(sorted(set(smiles)), 950)

33905
33742


In [173]:
### write files in alexandria1
outpath = "/alexandria1/acomajuncosa/MurD/ligand_preparation_Miquel/splits"
count_molecule = 0

for c, smi in tqdm.tqdm(enumerate(smiles)):
    
    # Create directories
    if os.path.exists(os.path.join(outpath, "smiles" + str(c))) is False:
        os.makedirs(os.path.join(outpath, "smiles" + str(c)))
    if os.path.exists(os.path.join(outpath, "out_smiles" + str(c))) is False:
        os.makedirs(os.path.join(outpath, "out_smiles" + str(c)))
        
    # Write files
    with open(os.path.join(outpath, "smiles" + str(c), "smiles" + str(c)), "w") as f:
        for s in smi:
            f.write("\t".join([s, str(count_molecule)]) + "\n")
            count_molecule += 1

950it [04:10,  3.79it/s]


In [24]:
### CHECK THAT MOST OF THEM ARE KEPT
path = '/alexandria1/acomajuncosa/MurD/ligand_preparation_Miquel/splits'
original_, final_ = [], []

for split in tqdm.tqdm(range(950)):
    original = open(os.path.join(path, "smiles" + str(split), "smiles" + str(split)), "r").readlines()
    final = [i.title for i in pybel.readfile("sdf", "/alexandria1/acomajuncosa/MurD/ligand_preparation_Miquel/splits/out_smiles" + str(split) + "/out_smiles" + str(split) + ".sdf")]
    original_.extend(original)
    final_.extend(final)
    # print(split, round(len(set(final))/len(set(original))*100, 1))

100%|██████████| 950/950 [03:25<00:00,  4.62it/s]


In [25]:
len(set(original_)), len(set(final_)), round(len(set(final_))/len(set(original_))*100, 1)

(33742, 30324, 89.9)

In [26]:
### COPY DATA TO SL

In [28]:
path = '/alexandria1/acomajuncosa/MurD/ligand_preparation_Miquel/splits'
outpath = '/slgpfs/projects/irb35/acomajuncosa/MurD/rest_docking/Miquel/structures'

for file in tqdm.tqdm(sorted(os.listdir(path))):
    if "out" in file:
        if os.path.exists(os.path.join(outpath, file)) is False: os.makedirs(os.path.join(outpath, file))
        shutil.copyfile(os.path.join(path, file, file + ".sdf"), os.path.join(outpath, file, file + ".sdf"))

100%|██████████| 1901/1901 [00:57<00:00, 32.82it/s]


In [168]:
### RUN DOCKINGS

In [32]:
path = '/slgpfs/projects/irb35/acomajuncosa/MurD/rest_docking/Miquel/structures'
elements = sorted(os.listdir(path))

In [33]:
len(elements)

950

In [16]:
elements = to_repeat

In [18]:
import sys
sys.path.insert(0, '/aloy/home/acomajuncosa/programs/hpc') #CHANGE THIS PATH TO YOUR HPC PATH!
from hpc import HPC
from starlife_config import config as cluster_config

scratch_path = "/slgpfs/scratch/irb35/acomajuncosa/MurD/rest_docking/Miquel_rep" 
script_path = "/slgpfs/projects/irb35/acomajuncosa/MurD/rest_docking/Miquel/gen_fps_rDock_center.py"

ncpus = 4
cluster = HPC(**cluster_config)
njobs = len(elements)

cluster_params = {}
cluster_params['job_name'] = 'murd_miq'
cluster_params["jobdir"] = scratch_path
cluster_params["memory"] = ncpus
cluster_params['cpu'] = ncpus
cluster_params["wait"] = False
cluster_params["elements"] = elements
cluster_params["num_jobs"] = len(elements)


singularity_image = "/slgpfs/projects/irb35/acomajuncosa/rDock_image_2.simg"
command = "singularity exec {} python {} <TASK_ID> <FILE>".format(
singularity_image,
script_path)

cluster.submitMultiJob(command, **cluster_params)

2022-06-28 10:51:25,901 chemicalchecker.util.config.config.Config [DEBUG   ] CC_CONFIG environment variable not set. Using default config file.
2022-06-28 10:51:25,901 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /home/acomajuncosa/programs/anaconda3/lib/python3.7/site-packages/chemicalchecker/util/config/cc_config.json
2022-06-28 10:51:26,976 hpc.HPC      [DEBUG   ] HPC system to use: slurm
2022-06-28 10:51:26,977 hpc.HPC      [DEBUG   ] initializing object slurm
2022-06-28 10:51:27,117 slurm.slurm  [DEBUG   ] Job nasdfdsfsdfsdfdme is: murd_miq
2022-06-28 10:51:27,131 slurm.slurm  [DEBUG   ] Num elements submitted 13
2022-06-28 10:51:27,131 slurm.slurm  [DEBUG   ] Num Job submitted 13
2022-06-28 10:51:27,138 slurm.slurm  [INFO    ] Writing file /slgpfs/scratch/irb35/acomajuncosa/MurD/rest_docking/Miquel_rep/job-murd_miq.sh...
2022-06-28 10:51:27,152 slurm.slurm  [DEBUG   ] HPC submission: sbatch --parsable /slgpfs/scratch/irb35/acomajuncosa/MurD/rest_docki

module 'matplotlib.font_manager' has no attribute '_rebuild'


2022-06-28 10:51:30,537 slurm.slurm  [DEBUG   ] load SINGULARITY/3


In [5]:
### LOOK FOR ERRORS/PROBLEMS

# SCRATCH
path = "/slgpfs/scratch/irb35/acomajuncosa/MurD/rest_docking/Miquel"
for file in tqdm.tqdm(sorted(os.listdir(path))):
    if '.out' in file:
        with open(os.path.join(path, file), "r") as f:
            for l in f:
                if "error" in l.lower() or "warning" in l.lower() or 'segmentation' in l.lower():
                    print(file)
                    break
                    


 11%|█         | 102/953 [00:00<00:01, 446.86it/s]

slurm-18106459_114.out
slurm-18106459_136.out


 61%|██████▏   | 584/953 [00:01<00:00, 439.72it/s]

slurm-18106459_557.out


 76%|███████▋  | 729/953 [00:01<00:00, 458.86it/s]

slurm-18106459_69.out


100%|██████████| 953/953 [00:02<00:00, 461.97it/s]

slurm-18106459_893.out
slurm-18106459_894.out
slurm-18106459_895.out


In [19]:
#RESULTS
path = "/slgpfs/projects/irb35/acomajuncosa/MurD/rest_docking/Miquel/structures"

original, final = set(), set()
to_repeat = []
data = {}

for split in tqdm.tqdm(sorted(os.listdir(path))):
    
    try:
    
        mymolecules = set([i.title for i in pybel.readfile("sdf", os.path.join(path, split, split + ".sdf"))])
        myfile = os.path.join(path, split, "rDock_results_" + split, "results.tar.gz")

        if os.path.exists(myfile):
            # if os.path.exists(os.path.join(path, split, "results", "results.sd")) is False:
            #     tar = tarfile.open(myfile, "r:gz")
            #     tar.extract("results/results.sd", os.path.join(path, split))
            myresults = set([i.title for i in pybel.readfile("sdf", os.path.join(path, split, "results", "results.sd"))])
            data[split] = myresults


            for i in mymolecules:
                original.add(i)
            for i in myresults:
                final.add(i)

            if len(myresults) < len(mymolecules)*0.5:
                print(split)
                to_repeat.append(split)

        else:
            print(split + " does not exist")
            
    except:
        
        print(split + ": some error...")
        
    # break

  7%|▋         | 69/950 [01:16<17:44,  1.21s/it]

out_smiles16


 12%|█▏        | 113/950 [02:10<14:34,  1.04s/it]

out_smiles20


 14%|█▍        | 136/950 [02:36<12:00,  1.13it/s]

...

 78%|███████▊  | 744/950 [13:05<01:43,  1.99it/s]

out_smiles769: some error...


 94%|█████████▍| 895/950 [15:20<00:21,  2.52it/s]

out_smiles901
out_smiles902: some error...
out_smiles903: some error...


100%|██████████| 950/950 [16:13<00:00,  1.02s/it]


In [20]:
len(original)

30264

In [21]:
len(final)

28537

In [22]:
original_ = []
final_ = []
for split in tqdm.tqdm(range(950)):
    try:
        original = [i.title for i in pybel.readfile("sdf", "/slgpfs/projects/irb35/acomajuncosa/MurD/rest_docking/Miquel/structures/out_smiles" + str(split) + "/out_smiles" + str(split) + ".sdf")]
        original_.extend(original)
        final = data["out_smiles" + str(split)]
        final_.extend(final)
    except:
        print(split)

 46%|████▌     | 434/950 [00:28<00:24, 21.42it/s]

432


 67%|██████▋   | 632/950 [00:41<00:18, 17.63it/s]

629


 81%|████████▏ | 774/950 [00:48<00:06, 27.40it/s]

769


 95%|█████████▌| 907/950 [00:55<00:02, 19.74it/s]

902
903


100%|██████████| 950/950 [00:57<00:00, 16.43it/s]


In [23]:
len(set(final_))

28537

In [25]:
len(set(original_))

30324